In [1]:
import os
from pathlib import Path
import glob
import pandas as pd

import numpy as np

In [3]:
os.chdir("..")

In [5]:
path = Path("data/GPTCloneBench/standalone/true_semantic_clones/py")
assert path.exists(), "Path does not exist"

# Types:
* Moderately type-3 (MT3) : similarity of 50%-70%
* Type-4 (T4)

* Prompt 1: Give me Type-3, Type-4 clone for the following code <code fragment>
* Prompt 2: Give me 10 distinctive implementation for the following code <code fragment>

In [57]:
import re

python_files = path.glob("**/*.py")
pattern = re.compile(r"data/GPTCloneBench/standalone/true_semantic_clones/py/(?P<prompt>.*?)/(?P<clonetype>.*?)/Clone_(?P<id>\d+)\.py")

clones = []
for file in python_files:
    match = pattern.match(str(file))
    if match:
        groups_dict = dict(match.groupdict())

        with open(file) as f:
            content = f.read()
        clone1,_, clone2 = content.partition("\n\n\n")
        groups_dict["clone1"] = clone1.strip().strip("\n")
        groups_dict["clone2"] = clone2.strip().strip("\n")
        groups_dict["file_path"] = file

        clones.append(groups_dict)
    else:
        print(f"no match for {file}")

In [58]:
df = pd.DataFrame(clones)
# df.set_index("id", inplace=True)
df.sort_values(by="id",inplace=True)

In [59]:
df.shape

(9447, 6)

In [60]:
df.head()

,prompt,clonetype,id,clone1,clone2,file_path
2116,prompt_2,MT3,1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers):\n\t\n if len(num...,data/GPTCloneBench/standalone/true_semantic_cl...
9361,prompt_1,T4,1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers) :\n\tsecond_larges...,data/GPTCloneBench/standalone/true_semantic_cl...
8682,prompt_1,MT3,1,def second_largest(numbers):\n count = 0\n ...,def second_largest(numbers):\n count = 0\n ...,data/GPTCloneBench/standalone/true_semantic_cl...
7187,prompt_2,T4,1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers):\n\n if len(numbe...,data/GPTCloneBench/standalone/true_semantic_cl...
2693,prompt_2,T4,10,"def num_input(prompt, error) :\n\twhile True :...","def num_input(prompt, error) :\n while ...",data/GPTCloneBench/standalone/true_semantic_cl...


In [61]:
df["clonetype"].value_counts()

clonetype
T4     6116
MT3    3331
Name: count, dtype: int64

#### Filter out MT3

In [62]:
df_t4 = df[df["clonetype"]=="T4"]

#### Checking out pairs

In [63]:
df_t4["clone2"].value_counts()

clone2
def unique(list):\n    s = {}\n    output = []\n    for x in list :\n        set_item_count(s, x)\n    for x in list :\n        count = reset_item_count(s, x)\n        if (count > 0) :\n            output.append(x)\n    return output                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     5
def fib(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a                                                                                                                      

In [64]:
valcnt = pd.DataFrame(df_t4["clone1"].value_counts(), index=None)
valcnt.reset_index(drop=False, inplace=True)
valcnt.head()

,clone1,count
0,def median(data) :\n\tnew_list = sorted(data)\...,75
1,"def merge(left_side, right_side) :\n\tresult =...",57
2,"def sublist(a, b) :\n\tlast = 0\n\tfor el_a in...",49
3,def unique(list) :\n\ts = {}\n\toutput = []\n\...,48
4,def median(x) :\n\tif len(x) % 2 == 0 :\n\t\tx...,43


In [65]:
df_t4[df_t4["clone1"]==valcnt.iloc[0]["clone1"]]

,prompt,clonetype,id,clone1,clone2,file_path
9015,prompt_1,T4,163,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n\tlist_length = len(data)\n...,data/GPTCloneBench/standalone/true_semantic_cl...
8814,prompt_1,T4,164,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n\tsorted_data = sorted(data...,data/GPTCloneBench/standalone/true_semantic_cl...
9406,prompt_1,T4,165,def median(data) :\n\tnew_list = sorted(data)\...,def median(data) : \n\tnew_list = sorted(data)...,data/GPTCloneBench/standalone/true_semantic_cl...
8856,prompt_1,T4,166,def median(data) :\n\tnew_list = sorted(data)\...,def median(data) :\n\tnew_list = sorted(data)\...,data/GPTCloneBench/standalone/true_semantic_cl...
9255,prompt_1,T4,167,def median(data) :\n\tnew_list = sorted(data)\...,def median(data) :\n\tnew_list = sorted(data)\...,data/GPTCloneBench/standalone/true_semantic_cl...
...,...,...,...,...,...,...
4568,prompt_2,T4,807,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n new_list = sorted(data)\n...,data/GPTCloneBench/standalone/true_semantic_cl...
3542,prompt_2,T4,808,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n sorted_list = sorted(data...,data/GPTCloneBench/standalone/true_semantic_cl...
3344,prompt_2,T4,809,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n newList = sorted(data)\n ...,data/GPTCloneBench/standalone/true_semantic_cl...
7034,prompt_2,T4,810,def median(data) :\n\tnew_list = sorted(data)\...,def median(data):\n sorted_list = sorted(data...,data/GPTCloneBench/standalone/true_semantic_cl...


In [66]:
df_t4[df_t4["clone1"]==valcnt.iloc[0]["clone1"]]["clone2"].value_counts()

clone2
def median(data):\n  length = len(data)\n  sorted_data = sorted(data)\n  if length % 2 == 0:\n    return (sorted_data[length // 2 - 1] + sorted_data[length // 2]) / 2.0\n  return sorted_data[length // 2]\n\n\ndef median(data):\n    lst = sorted(data)\n    mid = len(lst) // 2\n    if len(lst) % 2 == 0:\n        return (lst[mid] + lst[mid - 1]) / 2.0\n    else:\n        return lst[mid]    2
def median(data): \n\tdata.sort() \n\tlength = len(data)\n\tif length % 2 == 1 :  \n\t\treturn data[length // 2] \n\telse : \n\t\treturn (data[(length // 2) - 1] + data[length // 2]) / 2                                                                                                                                                                                                            1
def median(data):\n\tdata.sort()\n\tif len(data) % 2 == 0 :\n\t\thalf = int(len(data) / 2 - 1)\n\t\treturn (data[half] + data[half + 1]) / 2.0\n\telse:\n\t\treturn data[int(len(data) / 2)]                     

#### Cleaning

In [82]:
method_signature_pattern_at_start = re.compile(r'^def\s+\w+\s*\([\w,=\s*.\'\"-{}()]*\)\s*:\s*') 

def starts_with_method_signature(s):
    if method_signature_pattern_at_start.match(s):
        return True
    return False

In [76]:
df_t4[~df_t4["clone1"].map(lambda x: starts_with_method_signature(x))]

,prompt,clonetype,id,clone1,clone2,file_path


In [83]:
defects_df = df_t4[~df_t4["clone2"].map(lambda x: starts_with_method_signature(x))]
defects_df[["clone2", "file_path"]]

,clone2,file_path
3666,"#1 \ndef leap_years(start, end):\n result = 0...",data/GPTCloneBench/standalone/true_semantic_cl...
4156,"#7 \ndef leap_years(start, end):\n count = 0\...",data/GPTCloneBench/standalone/true_semantic_cl...
7528,"#6\ndef leap_years(start, end):\n result = ...",data/GPTCloneBench/standalone/true_semantic_cl...
5795,"#3\ndef leap_years(start, end):\n count = 0...",data/GPTCloneBench/standalone/true_semantic_cl...
4857,"#10 \ndef leap_years(start,end):\n result = 0...",data/GPTCloneBench/standalone/true_semantic_cl...
...,...,...
7264,"for root, _, files in os.walk('.'):\n f...",data/GPTCloneBench/standalone/true_semantic_cl...
7721,"os.chdir(folder_absPath)\n for root, _, fil...",data/GPTCloneBench/standalone/true_semantic_cl...
5840,#gpt output=============\n def median(midlist)...,data/GPTCloneBench/standalone/true_semantic_cl...
7450,return cache[i][j]\n \n return m...,data/GPTCloneBench/standalone/true_semantic_cl...


In [85]:
def clean_method_signature(s:str):
    if starts_with_method_signature(s):
        return s
    
    trash, middle, end = s.partition("def ")
    print("Removing: ", trash)

    return middle + end

In [86]:
df_t4["clone2"].map(lambda x: clean_method_signature(x))

Removing:  #1 

Removing:  #7 

Removing:  #6

Removing:  #3

Removing:  #10 

Removing:  #gpt output=============


Removing:  seq = list(seq)
	for index, ele in enumerate(seq):
		if pred(ele) is False:
			return seq[:index],seq[index:]
	return [],[]



Removing:  return before, after



Removing:  i +=1
  return seq[:i], seq[i:]



Removing:  #gpt output=============
 
Removing:  #gpt output=============
 
Removing:  close_bracket_count += 1
  
  return open_bracket_count == 0 and close_bracket_count > 0

 

Removing:  open_brackets -= 1
				close_brackets += 1
	
	return open_brackets == 0 and close_brackets > 0

 
Removing:  open_brackets -= 1
			if open_brackets<0:
				return False
	return open_brackets == 0

 
Removing:  #gpt output=============
 
Removing:  fhandle.close()
 
Removing:  open(fname, 'a').close()
 
Removing:  open(fname, 'a').close()
 
Removing:  return None

 

Removing:  return row, column
	return row,column

 

Removing:  #gpt output=============
 
Removing:  #gp

9361    def second_largest(numbers) :\n\tsecond_larges...
7187    def second_largest(numbers):\n\n  if len(numbe...
2693    def num_input(prompt, error) :\n        while ...
8843    def recurse(node, depth) :\n\tindent = "  " * ...
9418    def getName2(self, fullname) :\n\tnames = []\n...
                              ...                        
5605    def run(self) :\n    while True: \n        pre...
5566    def __init__(self, year = None, month = None, ...
2545    def __init__(self, year = None, month = None, ...
5232    def __init__(self, year = None, month = None, ...
2859    def __init__(self, year = None, month = None, ...
Name: clone2, Length: 6116, dtype: object

In [108]:
def_count_clone1 = pd.DataFrame(df_t4['clone1'].str.count('def'))
def_count_clone2 = pd.DataFrame(df_t4['clone2'].str.count('def'))
count_df = pd.concat([df_t4, def_count_clone1, def_count_clone2], axis=1)

# df_t4['def_count_clone1'] = df_t4['clone1'].str.count('def')
# df_t4['def_count_clone2'] = df_t4['clone2'].str.count('def')
count_df

,prompt,clonetype,id,clone1,clone2,file_path,def_count_clone1,def_count_clone2,clone1,clone2
9361,prompt_1,T4,1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers) :\n\tsecond_larges...,data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
7187,prompt_2,T4,1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers):\n\n if len(numbe...,data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
2693,prompt_2,T4,10,"def num_input(prompt, error) :\n\twhile True :...","def num_input(prompt, error) :\n while ...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
8843,prompt_1,T4,10,"def recurse(node, depth) :\n\tindent = "" "" * ...","def recurse(node, depth) :\n\tindent = "" "" * ...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
9418,prompt_1,T4,100,"def getName(self, name) :\n\tlisty = []\n\tspl...","def getName2(self, fullname) :\n\tnames = []\n...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5605,prompt_2,T4,995,def run(self) :\n\tt = datetime(* datetime.now...,def run(self) :\n while True: \n pre...,data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
5566,prompt_2,T4,996,"def __init__(self, year = None, month = None,d...","def __init__(self, year = None, month = None, ...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
2545,prompt_2,T4,997,"def __init__(self, year = None, month = None,d...","def __init__(self, year = None, month = None, ...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1
5232,prompt_2,T4,998,"def __init__(self, year = None, month = None,d...","def __init__(self, year = None, month = None, ...",data/GPTCloneBench/standalone/true_semantic_cl...,1,1,1,1


In [111]:
count_df[count_df["def_count_clone1"]>1]

,prompt,clonetype,id,clone1,clone2,file_path,def_count_clone1,def_count_clone2,clone1,clone2
4054,prompt_2,T4,1335,def curry(func) :\n def curried(*args) :\n ...,def curry(func) :\n def curried_func(*args)...,data/GPTCloneBench/standalone/true_semantic_cl...,3,4,3,4
5145,prompt_2,T4,1336,def curry(func) :\n def curried(*args) :\n ...,def curry(func) :\n def curried_func(*args)...,data/GPTCloneBench/standalone/true_semantic_cl...,3,4,3,4
3721,prompt_2,T4,1337,def curry(func) :\n def curried(*args) :\n ...,def curry(func) :\n def curried(*args) :\n ...,data/GPTCloneBench/standalone/true_semantic_cl...,3,3,3,3
5944,prompt_2,T4,1342,def curry(func) :\n def call_with(arg1) :\n ...,def curry(func):\n def call_with(*args):\n ...,data/GPTCloneBench/standalone/true_semantic_cl...,3,3,3,3
5622,prompt_2,T4,1343,def curry(func) :\n def call_with(arg1) :\n ...,def curry(func):\n def call_with(*args):\n ...,data/GPTCloneBench/standalone/true_semantic_cl...,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...
7450,prompt_2,T4,968,"def num_subsequences(seq, sub) :\n\tm, n = len...",return cache[i][j]\n \n return m...,data/GPTCloneBench/standalone/true_semantic_cl...,2,1,2,1
5754,prompt_2,T4,989,"def pop(self, key = None, default = object()) ...","def pop(self, key = None, default = object()) ...",data/GPTCloneBench/standalone/true_semantic_cl...,4,4,4,4
4463,prompt_2,T4,990,"def pop(self, key = None, default = object()) ...","def pop(self, key = None, default = object()) ...",data/GPTCloneBench/standalone/true_semantic_cl...,4,4,4,4
6232,prompt_2,T4,991,"def pop(self, key = None, default = object()) ...","def pop(self, key = None, default = object()) ...",data/GPTCloneBench/standalone/true_semantic_cl...,4,4,4,4


In [113]:
x ="""
def curry(func) :
    def curried(*args) :
            def curried_func(*more_args) :
                return func(*args, *more_args)
            return curried_func
    return curried


def curry(func) :
    def curried_func(*args) :
        if (len(args) == 0) :
            def inner(*_args) :
                return func(*_args)
            return inner
        else :
            def inner(*more_args) :
                given_args = list(args)
                given_args.extend(more_args)
                return func(*given_args)
            return inner
    return curried_func

"""

a = re.split(r"\n\n\n([#\w=\s]*\n\n?)?def",x)

In [120]:
a[1]

# Prepare Datasets

In [16]:
pairs = list(zip(df["clone1"], df["clone2"]))
print("All pairs",len(pairs))
# pairs[:2]
print("Dissimilar pairs", len(set(pairs)))

All pairs 9447
Dissimilar pairs 9365


In [17]:
len(df)

9447

In [18]:
len(df) / 2

4723.5

In [19]:
first_batch = df.iloc[:4723]
last_batch = df.iloc[-4723:]

first_batch.shape, last_batch.shape

((4723, 5), (4723, 5))

In [ ]:
dissimilar_pairs = []
for i in range(500):
    dissimilar_pairs.append((first_500.iloc[i]['clone1'], last_500.iloc[i]['clone1']))
    
for i in range(500):
    dissimilar_pairs.append((first_500.iloc[i]['clone2'], last_500.iloc[i]['clone2']))
    
dissimilar_pairs = []

for pair_1, pair_2 in zip(pairs[:500], pairs[500:]):
    dissimilar_pairs.append((pair_1[0], pair_2[0]))
    dissimilar_pairs.append((pair_1[1], pair_2[1]))
    
len(dissimilar_pairs)

In [24]:
df_t4.to_csv("data/gpt_benchmark_dataset_ONLY_T4.csv")

In [23]:
df.to_csv("data/gpt_benchmark_dataset.csv")